In [ ]:
""" Code to fit a pair-potential to reproduce a target equilibirum volume for diamond-structure Si;
    Note i just picked a target volume randomly - its not a physically meaningful value
"""

In [ ]:
import contextlib
import copy
import os
import itertools as it
import pathlib
from types import SimpleNamespace


#import matplotlib.pyplot as plt

import plato_pylib.shared.ucell_class as UCell
import plato_pylib.plato.mod_plato_inp_files as modInp
import plato_pylib.plato.parse_tbint_files as parseTbint
import plato_pylib.utils.job_running_functs as jobRun
import plato_pylib.utils.fit_eos as fitBMod

import plato_fit_integrals.core.coeffs_to_tables as coeffToTab
import plato_fit_integrals.core.create_analytical_reprs as analyticFuncts
import plato_fit_integrals.core.obj_funct_calculator as objCalculator
import plato_fit_integrals.core.opt_runner as optRun
import plato_fit_integrals.core.workflow_coordinator as wflowCoord

import plato_fit_integrals.initialise.create_coeff_tables_converters as createCoeffTabs
import plato_fit_integrals.initialise.fit_analytic_to_initial_tables as fitInit
import plato_fit_integrals.initialise.obj_functs_targ_vals as objTargs
import plato_fit_integrals.initialise.create_eos_workflows as eosFlow

import plato_fit_integrals.utils.plot_functs as fitPlotFuncts

MODEL_DATAFOLDER = "Test/mcweda"
MODEL_DATAFOLDER = "Test/format_4"
FULL_PATH_MODEL_DATAFOLDER = modInp.getAbsolutePathForPlatoTightBindingDataSet(MODEL_DATAFOLDER)

WORK_FOLDER = os.path.abspath("work_folder")
WORKFLOW_FOLDER = os.path.join(WORK_FOLDER,"calc_cubic_v0")
ATOM_SYMBOL = "Si"

TEST_VOL_PER_ATOM = [140, 150, 160, 170, 180, 190, 200, 210, 220]
K_POINTS = [10,10,10]

TARGET_V0 = 190 #Target value for the full primivitve cell
EOS="murnaghan"

FIT_PROPERTY = "v0"
FIT_STRUCTURE = "diamond"

NUMB_CORES = 6

PROG_STR = "tb1"

#Paramters for the analytic function we use to represent the pair-potential
RCUT = parseTbint.getBdtRcut( os.path.join(FULL_PATH_MODEL_DATAFOLDER,"{}_{}.bdt".format(ATOM_SYMBOL,ATOM_SYMBOL)) )
REF_R0 = 1.0
N_POLY = 3
TAIL_DELTA = 0.5





In [ ]:
#Create the initial structures

def createTestStructs(atomSymbol,testVols):
    fractCoords = [ [0.0, 0.0, 0.0, atomSymbol],
                    [0.25, 0.25, 0.25, atomSymbol] ]
    
    lattVects = [ [2.6954645, 2.6954645, 0.0      ],
                  [2.6954645, 0.0      , 2.6954645],
                  [0.0      , 2.6954645, 2.6954645] ]
    
    basicUCell = UCell.UnitCell.fromLattVects(lattVects, fractCoords = fractCoords)

    outCells = list()
    for currVol in testVols:
        currUCell = copy.deepcopy(basicUCell)
        currUCell.volume = currVol*len(fractCoords)
        outCells.append( currUCell )
    
    return outCells



In [ ]:
def createCoeffsToTablesObj():
    integHolder = createCoeffTabs.createIntegHolderFromModelFolderPath(FULL_PATH_MODEL_DATAFOLDER)
    integInfo = coeffToTab.IntegralTableInfo(FULL_PATH_MODEL_DATAFOLDER, "pairpot", ATOM_SYMBOL, ATOM_SYMBOL)
    relIntegTable = integHolder.getIntegTableFromInfoObj(integInfo,inclCorrs=False)
    aFunct = createAnalyticRepFunct(relIntegTable)
    return coeffToTab.CoeffsTablesConverter([aFunct], [integInfo], integHolder)
    

#Want the node to be movable
def createAnalyticRepFunct(integTable):
    nodePositions = fitInit.findCrossings(integTable.integrals)
    valAtR0 = fitInit.getInterpYValGivenXValandInpData(REF_R0,integTable.integrals)
    outFunct = analyticFuncts.Cawkwell17ModTailRepr_nodePosAsVars(rCut=RCUT,refR0=REF_R0,valAtR0=valAtR0,
                                                                 nPoly=N_POLY, tailDelta=TAIL_DELTA,
                                                                 nodePositions=nodePositions)
    print("nodePositions = {}".format(nodePositions))
    return outFunct
    
    


In [ ]:
def createObjFunctCalculator():
    targValue = TARGET_V0
    outCalculator = eosFlow.EosObjFunctCalculator.createEmptyInstance()
    outCalculator.addProp(FIT_STRUCTURE,FIT_PROPERTY,targValue)
    return outCalculator




In [ ]:
def createWorkflowToGetV0():
    structKey = FIT_STRUCTURE
    structs = {structKey:createTestStructs(ATOM_SYMBOL, TEST_VOL_PER_ATOM)} 
    optDict = dict()
    optDict["BlochStates".lower()] = K_POINTS
    optDict["dataset"] = MODEL_DATAFOLDER
    modOptsDicts = {structKey:optDict}
    nonE0EnergyDict = eosFlow.calcNonE0EnergyDict(structs, modOptsDicts, WORKFLOW_FOLDER, PROG_STR,nCores=NUMB_CORES)
    wFlow = eosFlow.CreateEosWorkFlow(structs, modOptsDicts, WORKFLOW_FOLDER, PROG_STR,eosModel=EOS,
                                      onlyCalcE0=True, nonE0EnergyDict=nonE0EnergyDict)()
    eosFlow.decorateEosWorkFlowWithPrintOutputsEveryNSteps(wFlow,printInterval=10)
    return wflowCoord.WorkFlowCoordinator([wFlow],nCores=NUMB_CORES)





In [ ]:
#Create the objective function for the main fit
coeffsToTablesObj = createCoeffsToTablesObj()
objFunctCalculator = createObjFunctCalculator()
workflowCoordForV0 = createWorkflowToGetV0()
mainObjFunct = optRun.ObjectiveFunction(coeffsToTablesObj, workflowCoordForV0, objFunctCalculator)

In [ ]:
#Fit coefficients to the initial pair-potential
fitResInitInts = fitInit.fitAnalyticFormToStartIntegrals(coeffsToTablesObj,method='Nelder-Mead')


In [ ]:
figA = fitPlotFuncts.plotFittedIntsVsInitial(coeffsToTablesObj._integInfo[0],coeffsToTablesObj)
figA.get_axes()[0].set_xlim(2,12)
figA.get_axes()[0].set_ylim(-0.4,1.0)

In [ ]:
coeffsToTablesObj._analyticalReps[0]

In [ ]:
#Run the main fit
mainFitRes = optRun.carryOutOptimisationBasicOptions(mainObjFunct,method='Nelder-Mead')

In [ ]:
mainFitRes

In [ ]:
#Fitted Vs orig.
figB = fitPlotFuncts.plotFittedIntsVsInitial(coeffsToTablesObj._integInfo[0],mainObjFunct.coeffTableConverter)
figB.get_axes()[0].set_xlim(2,12)
figB.get_axes()[0].set_ylim(-0.4,1.0)